# Uncertainty-based and random sampling strategies
This script run multiple classifiers in AL. You can specify the query strategy and float being used. 

In [ ]:
TRAIN_PATH = "../data/randomsplit/train"
TEST_PATH = "../data/randomsplit/test"

float_numbers = [
    '4903217',
    '4903218',
    '4903220', 
    '4903052',
    '4903054',
]
    
float_number = float_numbers[1]

QUERY_STRATEGY = 'random'
# QUERY_STRATEGY = 'uncertainty'

RESULT_PATH = f"../results/randomsplit/{float_number}/{QUERY_STRATEGY}"

import os
os.makedirs(RESULT_PATH, exist_ok=True)

n_initial = 1000
k = 1  # Number of samples to query at each iteration
budget = 100  # Number of queried samples desired

split_method = 'random'
# split_method = 'ocsvm'
# split_method = 'lof'

In [ ]:
RESULT_PATH

In [ ]:
def comp_ratio(dataset):
    ''' Compute anomaly ratio
    '''
    instance = dataset[(dataset['Label']==1)]
    rate=len(instance)/len(dataset)*100
    return round(rate,2), len(instance)

### Define model architectures

In [ ]:
import random
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score

from modAL.uncertainty import uncertainty_sampling

def create_model(model_name): 
    if model_name == 'KNN':
        model = KNeighborsClassifier(n_neighbors=5, leaf_size=30)
        model = RandomForestClassifier(n_estimators=20, random_state=42)
    elif model_name == 'XGBoost':
        model = XGBClassifier(max_depth=6)
    elif model_name == 'CatBoost':
        model = CatBoostClassifier(depth=2, iterations=20, silent=True)
    elif model_name == 'LightGBM':
        model = LGBMClassifier(max_depth=2, n_estimators=50)
    else:
        raise ValueError(f"Invalid model name: {model_name}")
    return model


def fit_model(model, labeled_data, model_name):
    X_train = labeled_data.drop(['ID', 'Label'], axis=1).values
    y_train = labeled_data['Label']
    
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, test_data, model_name):
    X_test = test_data.drop(['ID', 'Label'], axis=1).values
    y_test = test_data['Label'].values

    y_pred = model.predict(X_test)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    kappa = cohen_kappa_score(y_test, y_pred)
    return precision, recall, f1, kappa


### Define AL pipeline

In [ ]:
def query_strategy(strategy_name, model, X_unlabeled, k, model_name): 
    if strategy_name == 'uncertainty': 
        probabilities = model.predict_proba(X_unlabeled)
        # uncertainty_scores = uncertainty_sampling(model, X_unlabeled, n_instances=k)
        uncertainty_scores = 1 - (probabilities.max(axis=1))
         # Select the top-k most uncertain samples
        query_indices = (-uncertainty_scores).argsort()[:k]
    elif strategy_name == 'random': 
        num_rows = len(X_unlabeled)
        if num_rows == 0:
            raise ValueError("The matrix is empty.")
        if k > num_rows:
            raise ValueError("The number of rows to select is greater than the number of rows in the matrix.")
        query_indices = random.sample(range(num_rows), k)
    
    return query_indices

def pool_based_active_learning(model_name, initial_data, unlabeled_data, test_data, k, budget):
    model = create_model(model_name=model_name)
    labeled_data = initial_data.copy()  # Initialize the labeled set with the initial data

    queried_samples = 0
    query_indices = []
    query_ids = []

    # model_name = model.__class__.__name__
    metrics = {
        # 'model_name': model_name, 
        'num_samples': [], 
        'query_ids': [], 
        'Precision': [],
        'Recall': [],
        'F1-score': [],
        'Kappa': []
        }
    while queried_samples <= budget:
        # Iterate over the models
        # Train the model on the initial data
        model = fit_model(model, labeled_data, model_name)
        
        # Evaluate the model
        precision, recall, f1, kappa = evaluate_model(model, test_data, model_name)

        # Store the metrics for the current model
        metrics['num_samples'].append(queried_samples)
        metrics['query_ids'].append(query_ids)
        metrics['Precision'].append(precision)
        metrics['Recall'].append(recall)
        metrics['F1-score'].append(f1)
        metrics['Kappa'].append(kappa)
        
        # Compute uncertainty scores for the remaining unlabeled set
        # uncertainty_scores = -model.predict_proba(unlabeled_data.drop(['ID', 'Label'], axis=1).values).max(axis=1)
        X_unlabeled = unlabeled_data.drop(['ID', 'Label'], axis=1).values
        query_indices = query_strategy(QUERY_STRATEGY, model, X_unlabeled, k, model_name)

        # Add the queried samples to the labeled set
        # labeled_data = np.concatenate((labeled_data, unlabeled_data.iloc[query_indices]))
        labeled_data = pd.concat([labeled_data, unlabeled_data.iloc[query_indices]])
        query_ids = unlabeled_data.iloc[query_indices]['ID'].to_list()

        print(f"# samples: {queried_samples}; ID: {unlabeled_data.iloc[query_indices]['ID'].to_list()}; Label: {unlabeled_data.iloc[query_indices]['Label'].to_list()}")

        # Remove the queried samples from the unlabeled set
        unlabeled_data = unlabeled_data.drop(unlabeled_data.index[query_indices])

        # Update the number of queried samples
        queried_samples += len(query_indices)

        # # Train the final model on the labeled set
        # model = fit_model(model, labeled_data)

    return metrics

### Load data

In [ ]:
# %%capture
# Example usage
import os 
train_file = os.path.join(TRAIN_PATH, f'PR_PF_{float_number}.csv')
test_file = os.path.join(TEST_PATH, f'PR_PF_{float_number}.csv')
initial_file = os.path.join(TRAIN_PATH, f'{split_method}_PR_PF_{float_number}_{n_initial}_initial.csv')
unlabeled_file = os.path.join(TRAIN_PATH, f'{split_method}_PR_PF_{float_number}_{n_initial}_unlabeled.csv')

# Load the train and test datasets
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)
initial_data = pd.read_csv(initial_file)
unlabeled_data = pd.read_csv(unlabeled_file)

train_data = train_data.drop('Date', axis=1)
test_data = test_data.drop('Date', axis=1)
initial_data = initial_data.drop('Date', axis=1)
unlabeled_data = unlabeled_data.drop('Date', axis=1)


In [ ]:
print(f'------- {float_number} ------')
print(f'Train: {train_data.shape[0]}; {comp_ratio(train_data)[0]}%')
print(f'Test: {test_data.shape[0]}; {comp_ratio(test_data)[0]}%')
print(f"Initial: {n_initial}; {initial_data.Label.sum()} anomalies.")

### Start AL pipeline

In [ ]:
model_names = ['KNN', 'XGBoost', 'CatBoost', 'LightGBM']

# Dictionary to store the evaluation metrics for each model
metrics = {}

# Active learning loop
for model_name in model_names:
    metrics = pool_based_active_learning(model_name, initial_data, unlabeled_data, test_data, k, budget)
    df_metrics = pd.DataFrame(metrics)
    filename = f"{RESULT_PATH}/{model_name}_{split_method}_{n_initial}_initial_{k}_k.csv"
    df_metrics.to_csv(filename, index=False)
    print(f"Save to {filename}")
